One of your homework assignments was to write a method that finds twos. It takes two lists and returns any elements that appears inboth lists containing the digit 2.

Here is a pretty terrible implementation of `find_twos`:

In [129]:
def find_twos(first_list, second_list):
    return []

This is not a great implementation of `find_twos`, clearly :), but we can use it to write some tests and make sure that those tests pass and fail at the right times!

When we write the tests before the code, that is called **Test Driven Development (TDD)**.

How would we make sure that `find_twos` is working?

### This is one way:

In [1]:
def test_find_twos():
    assert(find_twos([], []) == [])
    assert(find_twos([2], [12]) == [])
    assert(find_twos([12], [12]) == [12])
    assert(find_twos([12, 2], [12]) == [12])
    assert(find_twos([12, 2, 3], [12, 2]) == [12, 2])
    assert(find_twos([12, 3], [12, 2]) == [12])
    assert(find_twos([1, 3, 4], [1, 3, 4]) == [])
    
test_find_twos()

AssertionError: 

**What are some of the problems with that way?**

### OK, how about this way? Is this better?

In [2]:
def test(function, examples):
    passed = 0
    run = 0

    for example in examples:
        run += 1
        expected = example[-1]
        actual = function(*example[:-1])

        if expected == actual:
            passed += 1
        else:
            print(f"Whoops. For example {example}, the function returned {actual}.")

    print(f"\n{passed} out of {run} examples worked as expected.")

find_twos_examples = [
    ("", "", []),
    ("1", "1, 3", []),
    ("2", "", []),
    ("2", "1, 3", []),
    ("2", "2", [2]),
    ("2", "12", []),
    ("12", "2, 12", [12]),
    ("1, 3, 5, 12, 7, 200", "2, 6, 9, 200, 5", [200]),
    ("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2", [2, 22]),
    ("1,2, 20,22, 44, 99", "3,5, 22, 100, 44, 2", [2, 22]),
    ("1,2, 20,22, 22,44, 20, 99", "3,5, 22, 100, 44, 2", [2, 22]),
    ("1, 2, 20, 22", "3, 2, 20, 22", [2, 20, 22]),
]

test(function=find_twos, examples=find_twos_examples)

Whoops. For example ('2', '2', [2]), the function returned [].
Whoops. For example ('12', '2, 12', [12]), the function returned [].
Whoops. For example ('1, 3, 5, 12, 7, 200', '2, 6, 9, 200, 5', [200]), the function returned [].
Whoops. For example ('1, 2, 20, 22, 44, 99', '3, 5, 22, 100, 44, 2', [2, 22]), the function returned [].
Whoops. For example ('1,2, 20,22, 44, 99', '3,5, 22, 100, 44, 2', [2, 22]), the function returned [].
Whoops. For example ('1,2, 20,22, 22,44, 20, 99', '3,5, 22, 100, 44, 2', [2, 22]), the function returned [].
Whoops. For example ('1, 2, 20, 22', '3, 2, 20, 22', [2, 20, 22]), the function returned [].

5 out of 12 examples worked as expected.


**What do you like about this way? What don't you like about this way?**

### Let's see if we can get this to work instead:

```
>>> assert_that(find_twos("", "")).equals([])
>>> True

>>> assert_that(find_twos("", "")).equals(["wrong answer"])
>>> FailedAssertion: Expected ['wrong answer'] but got []
```

### BREAKOUT GROUP TIME 🎉🎉🎉

Here's some code to get you started:

In [67]:
class FailedAssertion(Exception):
    pass

class Assertion:
    def __init__(self, expression):
        self.expression = expression

    def equals(self, expected_result):
        '''
            This method should return True if the assertion works
            And it should raise a FailedAssertion if it doesn't. GO!
        '''
        
def assert_that(expression):
    return Assertion(expression)

In [68]:
assert_that(find_twos("", "")).equals([])

True

In [69]:
assert_that(find_twos("", "")).equals(["wrong answer"])

FailedAssertion: Expected ['wrong answer'] but got []

The `assert_that` and `equals` methods that we have called above are called **matchers**. Test frameworks often have a wide variety of matchers that programmers like you or I can use to write expressions describing all the things we want to check in our tests. We will write more matchers a little later, but `.equals()` is a great one to start with.

We can use our `.equals()` matcher to write tests for `find_twos` like this:

In [84]:
    def test_empty_inputs(self):
        assert_that(find_twos("", "")).equals([])
        assert_that(find_twos("2", "")).equals([])
        assert_that(find_twos("2", "")).equals([])

    def test_non_matching_sets(self):
        assert_that(find_twos("1", "1, 3")).equals([])

    def test_non_matching_twos(self):
        assert_that(find_twos("2", "1, 3")).equals([])
        
    def test_matches(self):
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])

We could call each of the above methods individually:

In [81]:
tests = FindTwosTest()
test_empty_inputs()

In [83]:
test_non_matching_sets()

But that's kind of annoying. We don't want to have to explicitly _call_ every test that we write like this. It would be annoying to have to keep a file that just lists out and calls every single test method so we can run our tests all at once...plus, that's mistake-prone, since we might write a test and forget to put it in the list, or delete one and forget to take it out.

What if our `Test` superclass were able to run all our tests for us?

In [109]:
class Test():
    # Runs all the test methods. HOW?!?!
    def run(self):
        test_methods = [
            token for token in dir(self) \
            if token.startswith("test")  \
            and callable(getattr(self.__class__, token))
        ]
        for method in test_methods:
            print(f"Running {method}.")
            try:
                getattr(self.__class__, method).__call__(self)
            except Exception as e:
                print(e) 

**What is happening in this `run` method?**

1. "test_methods" is assignmed using a **list comprehension**
1. The backslashes in the list comprehension allow me to split what would be a very long line of Python into multiple lines for better legibility.

You should recognize:
1. The for loop
1. The f-string in the print statement

What are these things doing?
1. The `dir` method
1. The `.startswith()` method
1. The `.__class__` method
1. The `try` and `except` blocks
1. The `.__getattr__` method
1. The `callable` method
1. The `__call__` method

You will spend 90% of your programming time _reading_ code and the other 10% _writing_ code. So it is critical to practice _reading_ code and understanding what it is doing. Python provides you with some assistance for researching code that you are reading:

In [110]:
dir.__doc__

"dir([object]) -> list of strings\n\nIf called without an argument, return the names in the current scope.\nElse, return an alphabetized list of names comprising (some of) the attributes\nof the given object, and of attributes reachable from it.\nIf the object supplies a method named __dir__, it will be used; otherwise\nthe default dir() logic is used and returns:\n  for a module object: the module's attributes.\n  for a class object:  its attributes, and recursively the attributes\n    of its bases.\n  for any other object: its attributes, its class's attributes, and\n    recursively the attributes of its class's base classes."

In [111]:
help(dir)

Help on built-in function dir in module builtins:

dir(...)
    dir([object]) -> list of strings
    
    If called without an argument, return the names in the current scope.
    Else, return an alphabetized list of names comprising (some of) the attributes
    of the given object, and of attributes reachable from it.
    If the object supplies a method named __dir__, it will be used; otherwise
    the default dir() logic is used and returns:
      for a module object: the module's attributes.
      for a class object:  its attributes, and recursively the attributes
        of its bases.
      for any other object: its attributes, its class's attributes, and
        recursively the attributes of its class's base classes.



So the Test class is going to be our **superclass**. We can now **subclass** that Test class like so:

In [112]:
class FindTwosTest(Test):
    test_useless_attribute = None
    test_other_useless_attribute = None

    def test_empty_inputs(self):
        print("")
        assert_that(find_twos("", "")).equals([])
        assert_that(find_twos("2", "")).equals([])
        assert_that(find_twos("2", "")).equals([])

    def test_non_matching_sets(self):
        assert_that(find_twos("1", "1, 3")).equals([])

    def test_non_matching_twos(self):
        assert_that(find_twos("2", "1, 3")).equals([])
        
    def test_matches(self):
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])

In [113]:
FindTwosTest().run()

Running test_empty_inputs.

Running test_matches.
Expected [12] but got []
Running test_non_matching_sets.
Running test_non_matching_twos.


On _this particular_ code, I have kept it in small chunks inside of a REPL environment so that you can remove or change lines of code to investigate what they are doing. For example:

1. What happens if you remove the "f" from the front of the f-string?
1. What happens if you comment out `and callable(getattr(self.__class__, token))` in the list comprehension?
1. What happens if you remove the try/except above and just call `getattr(self.__class__, method).__call__(self)` right after the print statement?

### We have a test runner. Woo!

Now, things could be better about this test runner. 

**Challenge 1:** Get the test output to count up and print out passages and failures for the test class that you've run.

In [116]:
FindTwosTest().run()

Running test_empty_inputs.

Running test_matches.
Expected [12] but got []
Running test_non_matching_sets.
Running test_non_matching_twos.


**Challenge 2:** Get the test output to print in COLORS!

Below see a block of example code to help you get started on that:

In [121]:
import sys

# For this, we need to install a library.
# When code requires a library to do something, we call that a dependency.
!{sys.executable} -m pip install colorama 

from colorama import Fore, Back, Style 
print(Fore.RED + 'some red text') 
print(Fore.GREEN + 'and some green text') 
print(Back.YELLOW + 'you can also do backgrounds')
print(Style.RESET_ALL) 
print('back to normal now')

You should consider upgrading via the '/Users/chelseatroy/.pyenv/versions/3.9.0/bin/python3.9 -m pip install --upgrade pip' command.
some red text
and some green text
you can also do backgrounds

back to normal now


In [ ]:
FindTwosTest().run()

### Okay, so. 

We have talked about the two critical components of a test framework:
    
1. A **runner** (to run the tests)
1. Some **matchers** (to write expressive tests with useful failure messages)

Now let's get a little more sophisticated. We've got one matcher, and it's a pretty useful matcher, so we can write a lot of tests with it. But what might be helfpul is to have some more matchers to make our tests more clear and expressive. For example, what if we had an `is_empty` matcher? Then we could do:

In [123]:
class FindTwosTest(Test):

    def test_empty_inputs(self):
        print("")
        assert_that(find_twos("", "")).is_empty()
        assert_that(find_twos("2", "")).is_empty()
        assert_that(find_twos("2", "")).is_empty()

    def test_non_matching_sets(self):
        assert_that(find_twos("1", "1, 3")).is_empty()

    def test_non_matching_twos(self):
        assert_that(find_twos("2", "1, 3")).is_empty()
        
    def test_matches(self):
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])

**Challenge 3:** Implement the `is_empty` assertion.

Below see a block of example code to help you get started on that:

In [74]:
# You can technically have a class subclass itself. 
# I have done so here to avoid copy-pasting the implementation of Assertion over.
# We are now adding methods to the assertion class.
# In Swift, by the way, there's a keyword for doing exactly this: extension.
class Assertion(Assertion): 
    def is_empty():
        '''
            This method should return True if a collection has no items
            And it should raise a FailedAssertion if it doesn't. GO!
        '''
        pass

In [ ]:
FindTwosTest().run()

Good, good. Our test framework is coming along swimmingly. 

One thing to know about tests is, when we're testing collections, it is often helpful to make sure the colleciton is the right _size_ before we go into a bunch of complicated assertions about what's _in_ it. With a nice matcher, we could do that like:

In [126]:
class FindTwosTest(Test):

    def test_matches(self):
        assert_that(find_twos("12", "2, 12")).has_size(1)
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).has_size(2)
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])

**Challenge 4:** Implement the `has_size` assertion.

In [127]:
class Assertion(Assertion): 
    def has_size():
        '''
            This method should return True if a collection has the right number of items
            And it should raise a FailedAssertion if it doesn't. GO!
        '''
        pass

In [ ]:
FindTwosTest().run()

Let's do just one more today. One of the annoying things about our implementation right now is that `find_twos` has to return results in a specific _order_ for the assertion to pass. For example, if we check:

```
assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])
```

And `find_twos` returns `[22, 2]`, it will fail. 

What if we don't care about order? What if we just want to make sure all the right items are in the collection, regardless of order? Then we could do:

In [126]:
class FindTwosTest(Test):

    def test_matches(self):
        assert_that(find_twos("12", "2, 12")).has_size(1)
        assert_that(find_twos("12", "2, 12")).has_items(12)
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).has_size(2)
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).has_items(2, 22)

**Challenge 5:** Implement the `has_items` assertion.

NOTE THAT the implementation above doesn't require the items to be passed in as a collection. Instead, the programmer writing the test can just pass in as many items to this method as they want.

In [ ]:
class Assertion(Assertion): 
    def has_items():
        '''
            This method should return True if a collection has the right items
            And it should raise a FailedAssertion if it doesn't. GO!
        '''
        pass

In [ ]:
FindTwosTest().run()

In [ ]:
## Great Job!

Next week, we will make our test framework even more sophisticated. For example, we might address these questions:
    
- What if I have several different subclasses of `Test` and I want to run _all_ my tests at once?
- What if I have some code that I need to run before and after every test?
- What if I want to be able to skip a test, or only run specific tests according to 